# Read File
- https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do
- 2019년, 2020년 기간별 박스오피스 관객 수 상위 50개씩 DataFrame

In [254]:
%ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: CA2E-52A2

 C:\Users\User\Desktop\Viral_Effect\Analysis 디렉터리

2021-11-08  오후 11:27    <DIR>          .
2021-11-08  오후 11:27    <DIR>          ..
2021-11-08  오후 11:23    <DIR>          .ipynb_checkpoints
2021-11-08  오후 11:27            28,238 DF_Configuration.ipynb
2021-11-06  오후 09:04           373,658 KOBIS_2019.xlsx
2021-11-06  오후 09:07           350,418 KOBIS_2020.xlsx
2021-11-08  오후 09:14                65 README.md
2021-11-08  오후 11:27            32,166 Statistical_Analysis.ipynb
               5개 파일             784,545 바이트
               3개 디렉터리  42,837,454,848 바이트 남음


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
data_19 = pd.read_excel('KOBIS_2019.xlsx', sep='\t')[:50]
data_20 = pd.read_excel('KOBIS_2020.xlsx', sep='\t')[:50]
movie_df = pd.concat([data_19, data_20])
movie_df.reset_index(inplace = True)
movie_df.drop(['index', '순위'], axis = 1, inplace=True)

In [18]:
movie_df

,영화명,개봉일,매출액,매출액 점유율,관객수,스크린수,상영횟수,대표국적,국적,배급사
0,극한직업,2019-01-23,139651845516,0.073,16265618,2003,292584,한국,한국,(주)씨제이이엔엠
1,어벤져스: 엔드게임,2019-04-24,122182694160,0.064,13934592,2835,242001,미국,미국,월트디즈니컴퍼니코리아 유한책임회사
2,겨울왕국 2,2019-11-21,111596248720,0.058,13369064,2648,282557,미국,미국,월트디즈니컴퍼니코리아 유한책임회사
3,알라딘,2019-05-23,106955138359,0.056,12552283,1409,266469,미국,미국,월트디즈니컴퍼니코리아 유한책임회사
4,기생충,2019-05-30,85883963645,0.045,10085275,1948,192855,한국,한국,(주)씨제이이엔엠
...,...,...,...,...,...,...,...,...,...,...
95,검객,2020-09-23,1619781870,0.003,190237,832,21595,한국,한국,"(주)키위미디어그룹,오퍼스픽쳐스(유),(주)씨네필운"
96,조제,2020-12-10,1673427570,0.003,186647,1179,38291,한국,한국,워너브러더스 코리아(주)
97,사라진 시간,2020-06-18,1652522580,0.003,185802,874,20790,한국,한국,(주)에이스메이커무비웍스
98,밤쉘: 세상을 바꾼 폭탄선언,2020-07-08,1544025340,0.003,182026,882,27363,미국,"미국,캐나다",(주)홈초이스


# DataFrame Configuration

## Delete Feature
> 필요없는 Feature 삭제 
- 영화와 OTT 둘 다 해당되지 않으면 제거
- 없어도 되겠다 싶은 Feature 제거

In [19]:
def movie_columns(movie_df):
    col = list(movie_df.columns)
    for i in ['국적', '매출액 점유율', '상영횟수', '스크린수']:
        col.remove(i)
    movie_df = movie_df[col]
    
    return movie_df

In [20]:
movie_df = movie_columns(movie_df)

## 영화이름
- movie_name

> 겹치는 영화 -> 합치는 게 나을 것 같음
- 관객수 많이 차이 : 재개봉
- 관객수 조금 차이 : 12월에 개봉해서 해 넘어감

In [7]:
# 겹치는 이름들이 있음 -> 알라딘
movie_name = movie_df['영화명']
movie_name

0                극한직업
1          어벤져스: 엔드게임
2              겨울왕국 2
3                 알라딘
4                 기생충
           ...       
95                 검객
96                 조제
97             사라진 시간
98    밤쉘: 세상을 바꾼 폭탄선언
99                알라딘
Name: 영화명, Length: 100, dtype: object

In [8]:
# 겹치는 영화명이 5개 있음
sum(movie_name.value_counts()==2)

5

In [9]:
# 겹치는 영화 처리하는 함수
# 영화명, 개봉일, 대표국적, 배급사 : 관객수 많은 행 사용
# 매출액, 관객수 : 더해서 사용

def duplicate_movie(movie_df):
    # 겹치는 영화 확인
    movie_cnt = movie_df.loc[:,'영화명'].value_counts()
    num = 0
    dup_movie = []

    for i in movie_cnt:
        if i == 2:
            dup_movie.append(movie_cnt.index[num])
        num += 1
    
    # 관객수 적은 행 
    small_movie = []
    for name in dup_movie:
        small_view = (movie_df[movie_df['영화명'] == name].loc[:,'관객수'] > movie_df[movie_df['영화명'] == name].loc[:,'관객수'].mean())
        small_movie.extend(small_view.index[small_view])
        
    # 매출액, 관객수 합 사용
    for name in dup_movie:
        dup_movie_idx = movie_df[movie_df['영화명'] == name].index
        movie_df.loc[dup_movie_idx, '관객수'] = movie_df[movie_df['영화명']== name].loc[:,'관객수'].sum()
        movie_df.loc[dup_movie_idx, '매출액'] = movie_df[movie_df['영화명']== name].loc[:,'매출액'].sum()
        
    # 관객수 적은 행 삭제
    movie_df = movie_df.drop(small_movie)

    return movie_df

In [10]:
movie_df = duplicate_movie(movie_df)

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [11]:
# 겹치는 영화 있나 확인
movie_df.loc[:,'영화명'].value_counts()

조제                    1
남산의 부장들               1
나의 특별한 형제             1
82년생 김지영              1
겨울왕국 2                1
                     ..
미스터 주: 사라진 VIP        1
스타워즈: 라이즈 오브 스카이워커    1
히트맨                   1
주먹왕 랄프 2: 인터넷 속으로     1
내안의 그놈                1
Name: 영화명, Length: 95, dtype: int64

In [12]:
movie_df.loc[movie_df['영화명'] == '알라딘',]

,영화명,개봉일,매출액,관객수,대표국적,배급사
99,알라딘,2019-05-23,108160549039,12723777,미국,월트디즈니컴퍼니코리아 유한책임회사


In [13]:
# 중복되는 영화 제거 후 영화명
movie_name = list(movie_df.loc[:, '영화명'])
print(movie_name)

['극한직업', '어벤져스: 엔드게임', '엑시트', '스파이더맨: 파 프롬 홈', '캡틴 마블', '조커', '봉오동 전투', '라이온 킹', '나쁜 녀석들: 더 무비', '82년생 김지영', '분노의 질주: 홉스&쇼', '토이 스토리 4', '돈', '악인전', '가장 보통의 연애', '말모이', '증인', '블랙머니', '터미네이터: 다크 페이트', '사바하', '타짜: 원 아이드 잭', '신의 한 수: 귀수편', '알리타: 배틀 엔젤', '내안의 그놈', '뺑반', '변신', '주먹왕 랄프 2: 인터넷 속으로', '걸캅스', '사자', '아쿠아맨', '드래곤 길들이기 3', '나의 특별한 형제', '어스', '말레피센트 2', '유열의 음악앨범', '퍼펙트맨', '포드 V 페라리', '생일', '힘을 내요, 미스터 리', '항거:유관순 이야기', '장사리 : 잊혀진 영웅들', '마이펫의 이중생활 2', '롱 리브 더 킹: 목포 영웅', '쥬만지: 넥스트 레벨', '존 윅 3: 파라벨룸', '남산의 부장들', '다만 악에서 구하소서', '반도', '히트맨', '테넷', '백두산', '#살아있다', '강철비2: 정상회담', '담보', '닥터 두리틀', '삼진그룹 영어토익반', '정직한 후보', '도굴', '클로젯', '오케이 마담', '해치지않아', '천문: 하늘에 묻는다', '결백', 1917, '작은 아씨들', '미드웨이', '시동', '지푸라기라도 잡고 싶은 짐승들', '미스터 주: 사라진 VIP', '인비저블맨', '나쁜 녀석들: 포에버', '국제수사', '침입자', '스타워즈: 라이즈 오브 스카이워커', '스파이 지니어스', '이웃사촌', '온워드: 단 하루의 기적', '소리도 없이', '버즈 오브 프레이(할리 퀸의 황홀한 해방)', '원더 우먼 1984', '겨울왕국 2', '오! 문희', '그린랜드', '위대한 쇼맨', '런', '뮬란', '내가 죽던 날', '기생충', '신비아파트 극장판 하늘도깨비 대 요르문간드', '프리즌 이스케이프', '

# Processed DataFrame 

In [68]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [69]:
data_19 = pd.read_excel('KOBIS_2019.xlsx', sep='\t')[:50]
data_20 = pd.read_excel('KOBIS_2020.xlsx', sep='\t')[:50]
movie_df = pd.concat([data_19, data_20])
movie_df.reset_index(inplace = True)
movie_df.drop(['index', '순위'], axis = 1, inplace=True)

In [70]:
def movie_columns(movie_df):
    col = list(movie_df.columns)
    for i in ['국적', '매출액 점유율', '상영횟수', '스크린수']:
        col.remove(i)
    movie_df = movie_df[col]
    
    return movie_df

In [71]:
def duplicate_movie(movie_df):
    # 겹치는 영화 확인
    movie_cnt = movie_df.loc[:,'영화명'].value_counts()
    num = 0
    dup_movie = []

    for i in movie_cnt:
        if i == 2:
            dup_movie.append(movie_cnt.index[num])
        num += 1
    
    # 관객수 적은 행 
    small_movie = []
    for name in dup_movie:
        small_view = (movie_df[movie_df['영화명'] == name].loc[:,'관객수'] > movie_df[movie_df['영화명'] == name].loc[:,'관객수'].mean())
        small_movie.extend(small_view.index[small_view])
        
    # 매출액, 관객수 합 사용
    for name in dup_movie:
        dup_movie_idx = movie_df[movie_df['영화명'] == name].index
        movie_df.loc[dup_movie_idx, '관객수'] = movie_df[movie_df['영화명']== name].loc[:,'관객수'].sum()
        movie_df.loc[dup_movie_idx, '매출액'] = movie_df[movie_df['영화명']== name].loc[:,'매출액'].sum()
        
    # 관객수 적은 행 삭제
    movie_df = movie_df.drop(small_movie)

    return movie_df

In [72]:
def df_reset_index(movie_df):
    movie_df.reset_index(inplace=True)
    movie_df.drop(['index'], inplace = True, axis = 1)
    return movie_df

In [73]:
movie_df = movie_columns(movie_df)
movie_df = duplicate_movie(movie_df)
movie_df = df_reset_index(movie_df)

In [74]:
movie_df

,영화명,개봉일,매출액,관객수,대표국적,배급사
0,극한직업,2019-01-23,139651845516,16265618,한국,(주)씨제이이엔엠
1,어벤져스: 엔드게임,2019-04-24,122182694160,13934592,미국,월트디즈니컴퍼니코리아 유한책임회사
2,엑시트,2019-07-31,79232012162,9426011,한국,(주)씨제이이엔엠
3,스파이더맨: 파 프롬 홈,2019-07-02,69010000100,8021145,미국,소니픽쳐스엔터테인먼트코리아주식회사극장배급지점
4,캡틴 마블,2019-03-06,51507488723,5802810,미국,월트디즈니컴퍼니코리아 유한책임회사
...,...,...,...,...,...,...
90,검객,2020-09-23,1619781870,190237,한국,"(주)키위미디어그룹,오퍼스픽쳐스(유),(주)씨네필운"
91,조제,2020-12-10,1673427570,186647,한국,워너브러더스 코리아(주)
92,사라진 시간,2020-06-18,1652522580,185802,한국,(주)에이스메이커무비웍스
93,밤쉘: 세상을 바꾼 폭탄선언,2020-07-08,1544025340,182026,미국,(주)홈초이스
